# Interpolation search
### Introduction
The background to the interpolation search is described in **MOD007357 Coursework Brief 2021-22 (Task a)**, as search algorithm for sorted arrays to find the position of a search value. The task is to compare search performance for three strategies:

1. Pure interpolation search
2. Mixed interpolation search and binary search
3. Pure binary search

### Implementation summary
The seearch algorithms have been implemented in the *python* programming language, and can be applied to an input array by initialising an `ArraySearcher` class with the array, and calling the class' `search()` method - providing a search strategy input.

### ZIP archive
The code output of running the `main()` method can be found in the associated ZIP archive (`interpolation-search.zip`). Contents:

```
interpolation-search.zip
  - Interpolation search.html
  - Interpolation search.ipynb
  - interpolation_search.py
  - README.md
  - environment.yml
  results/
    - comparison_pandas.png
    - testing_df.csv
```

### Dependencies
The `conda` environment required to run the `interpolation_search.py` code, and this notebook, can be recreated from the *environment.yml* file in the ZIP archive, and activated as follows:

```bash
conda env create -f environment.yml
conda activate interpolation-search
```

We can use a jupyter 'magic' command to generate the *environment.yml* file.

In [1]:
!conda env export --from-history > environment.yml

### Notebook
This notebook was created within the same python `conda` environment used to run the `interpolation_search.py`.

To generate the PDF version, the notebook was exported to HTML format and then printed to PDF format from Google Chrome.

### Imports

In [2]:
from interpolation_search import ArraySearcher
from interpolation_search import generate_random_array, get_random_array_item

### Walkthrough examples

An example of each of our 3 searching algorithms can be seen below for a single input array.

#### Input parameters

In [3]:
MIN_VAL = 0
MAX_VAL = 1000
CARDINALITY = 50

#### Generate the array
The `ArraySearcher` class will ensure any array passed to it is sorted before applying a search algorithm.

In [4]:
array = generate_random_array(MIN_VAL, MAX_VAL, CARDINALITY)
print(sorted(array))

[0, 3, 25, 42, 45, 51, 118, 125, 193, 235, 252, 255, 322, 324, 341, 356, 406, 413, 437, 479, 565, 591, 602, 614, 619, 631, 652, 652, 660, 666, 727, 727, 730, 734, 759, 761, 770, 772, 774, 775, 784, 823, 858, 868, 903, 905, 942, 972, 979, 985]


#### Generate query value

In [5]:
query_val = get_random_array_item(array)
print(query_val)

775


#### Apply each search algorithm in turn

In [6]:
searcher = ArraySearcher(array)

In [10]:
search_counts = []
methods = []
for n in range(10000):
    factor = 10
    cardinality = CARDINALITY + (factor * n) 
    array = generate_random_array(MIN_VAL, cardinality * 10, cardinality)
    query_val = get_random_array_item(array)
    searcher = ArraySearcher(array)
    strategies = ['interpolation', 'mixed', 'binary']
    for strategy in strategies:
        searcher.search(query_val, search_strategy=strategy)
        search_counts.append(searcher.search_count)
        methods.append(strategy)
    print(array)

Iteration:   0 | Starting info        | Query value: 36 | Bottom index:  0 | Top index: 49 | Bottom value:   9 | Top value: 488
Iteration:   1 | Interpolation search | Query value: 36 | Bottom index:  0 | Top index: 49 | Bottom value:   9 | Top value: 488
Index guess 2 value (25) is too low.
Percentage of array eliminated: 6%
Iteration:   2 | Interpolation search | Query value: 36 | Bottom index:  3 | Top index: 49 | Bottom value:  29 | Top value: 488
Index guess 3 value (29) is too low.
Percentage of array eliminated: 2%
Iteration:   3 | Interpolation search | Query value: 36 | Bottom index:  4 | Top index: 49 | Bottom value:  36 | Top value: 488
The query value (36) was found at index 4 (of the sorted array) after 3 iteration(s)
Iteration:   0 | Starting info        | Query value: 36 | Bottom index:  0 | Top index: 49 | Bottom value:   9 | Top value: 488
Iteration:   1 | Interpolation search | Query value: 36 | Bottom index:  0 | Top index: 49 | Bottom value:   9 | Top value: 488
Ind

In [16]:
len(search_counts)
len(methods)

for idx, method in enumerate(strategies):
    print(method)
    for position in range(idx, len(search_counts), 3):
        print(search_counts[position])


interpolation
3
2
2
2
5
5
2
4
2
2
3
3
1
5
3
2
1
1
3
3
3
3
3
2
2
4
3
6
4
4
3
1
1
4
2
1
3
3
2
3
4
5
1
3
5
2
4
3
1
3
2
2
4
2
5
2
2
4
3
2
3
3
3
1
2
2
3
3
3
4
2
6
3
2
3
3
3
4
3
3
4
5
3
2
3
4
4
2
3
3
1
4
4
3
4
1
3
4
3
5
mixed
5
3
2
2
8
5
2
4
3
3
3
3
1
5
4
2
1
1
3
4
5
5
7
2
2
7
4
9
6
5
5
1
1
7
3
1
5
3
2
4
6
8
1
4
6
2
8
3
1
4
3
3
5
2
6
4
2
6
5
2
3
4
3
1
3
2
3
3
4
6
2
10
4
2
3
3
3
6
4
4
6
9
3
4
3
4
4
3
4
3
1
4
6
4
5
1
5
6
3
5
binary
5
2
3
4
5
6
6
6
6
7
6
7
7
7
7
7
7
4
7
4
4
7
5
7
7
7
6
7
4
8
8
8
8
8
5
8
8
8
7
8
8
8
8
8
8
7
8
8
8
8
7
9
8
8
9
8
8
8
8
8
2
9
9
9
6
8
9
9
9
8
7
9
7
9
6
8
8
4
9
9
9
9
7
9
9
9
9
9
4
7
9
9
9
9
7
9
6
7
9
10


In [ ]:
()